In [280]:
print "OK"

"hello"
%load_ext autoreload
#%autoreload 2
import pandas 
import pandas as pd 
import sklearn
from backend import db_connection2 as db
from backend import db_connection2 
import sys

OK
The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [281]:
conn, meta, session = db.connect("postgres", "1234", db="match_stick_girl") #temp2

In [282]:
all_text_manipulations=session.query(db.Process_Text_Manipulation).all()

#baby_born
#babay
#rewrite_task2

#For all Process Rewrite
#For all COntent whose origin id == Process_Rewrite
#Where User_ID is > -1 #not null

#For process - hash_id process and estimated length 
#For content compeltion

In [289]:
one_el=all_text_manipulations[0]
#one_el=db_connection2.Process_Text_Manipulation
#one_el.task_parameters_obj

def time_estimate(process):
    total_text=""
    total_text+=" "+process.task_parameters_obj.body_of_task.results
    total_text+=" "+process.task_parameters_obj.prompt.results
    total_text+=" "+process.task_parameters_obj.result.results
    total_text+=" "+process.task_parameters_obj.suggestion.results
    total_text+=" "+process.task_parameters_obj.context.results
    words=len(total_text.split(" "))
    time = words/ 5.5

    return time  

print time_estimate(all_text_manipulations[3])

20.7272727273


In [290]:
#estimate rating score
#estimate writing score 
rating_score_matrix=[]
writing_score_matrix=[] #writing score matrix first
databases=["match_stick_girl","baby_born","babay"]

In [291]:
for database in databases:
    conn, meta, session = db.connect("postgres", "1234", db=database) 
    
    process=session.query(db.Process_Text_Manipulation).subquery()
    content=session.query(db.Content).filter((process.c.id == 
                                             db.Content.origin_process_id)
                                             & (db.Content.is_completed==True)
                                             )
    content_subquery=content.subquery()
    
    
    task_params=session.query(db.Task_Parameters).filter\
        (content_subquery.c.id==db.Task_Parameters.result_id).subquery()
    
    process_ratings=session.query(db.Process_Rate).\
        filter(task_params.c.id==db.Process_Rate.task_parameters_id).all()
    
    #Time it took to read the task, time it took to complete, and overall score
    for i in process_ratings:
        if  len(i.get_final_results_complete(session).all()) > 0:

            
            result=i.task_parameters_obj.result 
    
            
            score= i.get_final_results_complete(session).all()[0].results 
            
            associated_process=session.query(db.Process_Text_Manipulation).filter\
                (result.origin_process_id==db.Process_Text_Manipulation.id).all()[0]
            time_to_read= time_estimate(associated_process)
            time_to_complete= (result.completed_date-result.assigned_date).total_seconds()
            
            print i.task_parameters_obj.body_of_task.results 
            print i.task_parameters_obj.prompt.results 
            
            print score 
            print time_to_complete
            print time_to_read
            print i.task_parameters_obj.result.results 
            print "next"
            
            writing_score_matrix.append(
                [float(score),time_to_read,time_to_complete])
            for j in i.get_content_produced_by_this_process():
                if j.is_completed == True:
                    time_to_read= time_estimate(i) 
                    avg_score=float(score)
                    score = -2
                    try:
                        score=float(j.results)
                    except: pass 
                    general_time_to_complete=time_to_complete 
                    time_to_complete=(j.completed_date-j.assigned_date).total_seconds()
                    time_to_read_result=len(result.results.split(" "))/5.5
                    rating_task=[score,avg_score,time_to_read,time_to_read_result,
                                 time_to_complete]
                    rating_score_matrix.append(rating_task)
        #print "end of content"

 They were very large slippers, which her mother had hitherto worn. so large were they. and the poor little thing lost them as she scuffled away across the street, because of two carriages that rolled by dreadfully fast.
One slipper was nowhere to be found. the other had been laid hold of by an urchin, and off he ran with it
The 'content to be evaluated' is intended to be a summary of the text below. Rate how well you feel it accurately captures the full meaning and key events. 
4.0
151.253226
16.5454545455
The mother lost her slippers because they were so big they fell off as she crossed the street as two carriages came by quickly.  An urchin stole one and the other was lost.
next
 he thought it would do capitally for a cradle when he some day or other should have children himself. So the little maiden walked on with her tiny naked feet, that were quite red and blue from cold. She carried a quantity of matches in an old apron, and she held a bundle of them in her hand. Nobody had boug


The 'content to be evaluated' is intended to be a summary of the text below. Rate how well you feel it accurately captures the full meaning and key events. 
3.0
34.480603
18.5454545455
The girl is cold and is trying to get warm with fire and matches.
next

She rubbed another against the wall: it burned brightly, and where the light fell on the wall, there the wall became transparent like a veil, so that she could see into the room. On the table was spread a snow-white tablecloth. upon it was a splendid porcelain service, and the roast goose was steaming famously with its stuffing of apple and dried plums. And what was still more capital to behold was, the goose hopped down from the dish, reeled about on the floor with knife and fork in its breast, till it came up to the poor little girl. when--the match went out and nothing but the thick, cold, damp wall was left behind
The 'content to be evaluated' is intended to be a summary of the text below. Rate how well you feel it accurately ca


The 'content to be evaluated' is intended to be a summary of the text below. Rate how well you feel it accurately captures the full meaning and key events. 
5.0
125.145003
33.0909090909
A poor girl was out on the streets and was very cold. She was selling matches to earn money for her father. She didn't want to go home because she hadn't sold any matches so her father would beat her. She knew she shouldn't use the matches but she takes one out and lights it. She enjoys its warmth.
next
Most terribly cold it was. it snowed, and was nearly quite dark, and evening-- the last evening of the year. In this cold and darkness there went along the street a poor little girl, bareheaded, and with naked feet. When she left home she had slippers on, it is true. but what was the good of that
The 'content to be evaluated' is intended to be a summary of the text below. Rate how well you feel it accurately captures the full meaning and key events. 
4.0
66.679439
15.8181818182
It was snowy, dark and to


The 'content to be evaluated' is intended to be a summary of the text below. Rate how well you feel it accurately captures the full meaning and key events. 
1.0
87.587593
16.3636363636
She sat there with her matches, an entire bundle burnt. People just said, "She wanted to warm herself." But no one really knew what she had seen and no one could ever dream of the splendor in which, with her grandmother, she had entered into a new year.
next
 "Oh, take me with you! You go away when the match burns out. you vanish like the warm stove, like the delicious roast goose, and like the magnificent Christmas tree!" And she rubbed the whole bundle of matches quickly against the wall, for she wanted to be quite sure of keeping her grandmother near her. And the matches gave such a brilliant light that it was brighter than at noon-day: never formerly had the grandmother been so beautiful and so tall. She took the little maiden, on her arm, and both flew in brightness and in joy so high, so very high


Looking at the text BELOW and the text on the LEFT, please rate how well 'content to be evaluate' incorporates them both into a single summary
3.0
116.262802
19.2727272727
A cold and hungry woman was out on the streets on New Years Eve. She was selling matches to earn money for her father. But she hadn't sold any matches and didn't want to go home because her father would beat her. She knew that she shouldn't waste the matches but she lit one anyway, enjoying the small warmth it brings.
next
It was snowy, dark and toward the end of the day on New Year's Eve. A poor girl on the streets left home.
Looking at the text BELOW and the text on the LEFT, please rate how well 'content to be evaluate' incorporates them both into a single summary
1.8
111.348442
17.0909090909
On a dark and snowy evening, a poor girl left her home to wander the streets. It was New Year's Eve and her mother was concerned that the girl would freeze to death. The family did not have enough money to buy very many thin

...


Rate how well the ideas listed below would improve the content
3.0
38.803022
16.0
Happy belated 30th birthday son!  How does it feel that you're no longer a millennial?

Love,

Dad
next
sorry about that so Happy 30th. No longer a millennial and what does it make you. love, dad 
Rate how well the text below improves and expands the content. Uses the suggestions and context to help inform you
3.33333333333
34.343653
18.1818181818
Happy 30th! You'll always be a millennial to me. Love you!
next
sorry about that so Happy 30th. No longer a millennial and what does it make you. love, dad 
Rate how well the text below improves and expands the content. Uses the suggestions and context to help inform you
3.66666666667
234.317467
18.1818181818
Happy belated birthday, son! I'm sorry I couldn't write this sooner. Wow, you've just turned thirty. How does it feel to no longer be a millennial? Don't sweat it, look at me! I'm old and am still love life. But this day is about you. Keep your spirits up.


Rate how well the text below improves and expands the content. Uses the suggestions and context to help inform you
4.33333333333
59.320906
18.1818181818
Son, 

Sorry about absentmindedly overlooking your birthday.  I suppose that's going to happen more and more as I age.  So don't be surprised.  Anyway, Happy 30th!  Now that you aren't a millenial anymore, what exactly are you?

Love,  Dad
next
I had just turned 30 and was feeling a bit down. To make matter worse, my Dad forgot my birthday and  sent me a short message saying "Happy Belated 30th birthday son! How does it feel that you're no longer a millenial? Love, Dad". 
Rate how well the ideas listed below would improve the content
3.33333333333
123.904253
13.4545454545
To make matters worst
To make matters worst, my dad
a short message saying, "Happy 
next
I had just turned 30 and was feeling a bit down. To make matter worse, my Dad forgot my birthday and  sent me a short message saying "Happy Belated 30th birthday son! How does it


Rate how well the ideas listed below would improve the content
3.66666666667
94.589113
13.4545454545
I would just say go out and have a few drinks with friends and just focus on what you will do now that your are 30
next
I had just turned 30 and was feeling a bit down. To make matter worse, my Dad forgot my birthday and  sent me a short message saying "Happy Belated 30th birthday son! How does it feel that you're no longer a millenial? Love, Dad". 
Rate how well the ideas listed below would improve the content
3.0
67.059218
13.4545454545
I am glad you're my son! 
I hope you are doing well today son, I love you.
Son, you are the world to me, I love you. Thank you for being you!
next
I had just turned 30 and was feeling a bit down. To make matter worse, my Dad forgot my birthday and  sent me a short message saying "Happy Belated 30th birthday son! How does it feel that you're no longer a millenial? Love, Dad". 
Rate how well the text below improves and expands the content. Uses the sugg


Rate how well the text below improves and expands the content. Uses the suggestions and context to help inform you
1.66666666667
61.641412
21.0909090909
I hope you will go out and enjoy your day with your friends.  Then tomorrow start thinking about the future.
next
I had just turned 30 and was feeling a bit down. To make matter worse, my Dad forgot my birthday and  sent me a short message saying "Happy Belated 30th birthday son! How does it feel that you're no longer a millenial? Love, Dad". 
Rate how well the text below improves and expands the content. Uses the suggestions and context to help inform you
4.33333333333
25.431828
21.0909090909
Happy Belated 30th birthday son! How does it feel that you're no longer a millenial
next
In Boston, there's a monument for the man who was the owner of one of the largest film production companies ever. In truth, he wasn't really the most amazing producer at all. He stole innocence and virtue from many women. While the man's monument still stand


Rate how well does the text below sound better than it's source material above. Also account for how well it incorporates the ideas and suggestions on the left
-2.0
91.347454
27.2727272727

next
In Boston, there's a monument for the man who was the owner of one of the largest film production companies ever. In truth, he wasn't really the most amazing producer at all. He stole innocence and virtue from many women. While the man's monument still stands, how many people today know that these women were traumatized? Unfortunately, the monument is made from materials that last, and the lie has stood the test of time. This example shows you have to be vigilant with regards to the truth, especially when the lie itself is weak. These weak lies won't last more than the average truths
Rate how well does the text below sound better than it's source material above. Also account for how well it incorporates the ideas and suggestions on the left
-2.0
71.593353
27.2727272727

next
In Boston, there's

In [271]:
import pandas as pd 
import numpy as np 
ary=np.array(writing_score_matrix)
frame=pd.DataFrame(data=ary,columns=["Score","Time_To_Read","Time_To_Complete"])
frame["ratio"]=frame.Time_To_Complete/frame.Time_To_Read
function=lambda t: "Bad" if t < 3 else "Good"
frame["binary_score"]=frame['Score'].apply(function)

In [272]:
from sklearn.linear_model import LogisticRegression
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import confusion_matrix
from sklearn import preprocessing
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import roc_curve

names=preprocessing.LabelEncoder()
names.fit(frame['binary_score'])

LabelEncoder()

In [273]:
from sklearn.metrics import r2_score
from sklearn import tree 
from sklearn.linear_model import Ridge
from sklearn.preprocessing import PolynomialFeatures
from sklearn.pipeline import make_pipeline
from sklearn.metrics import mean_absolute_error
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
#r=tree.DecisionTreeClassifier(criterion="entropy",min_samples_split=10)
#r=RandomForestClassifier(min_samples_split=10)
r=LogisticRegression()
#r=KNeighborsClassifier(n_neighbors=5)
p=make_pipeline( r) #PolynomialFeatures(1)

#p=LogisticRegression()

X=frame[["Time_To_Read","Time_To_Complete","ratio"]]
y=frame["binary_score"]
X, y = balanced_subsample(X.values,names.transform(y))
p.fit(X,y)

frame_pos=frame #frame[frame.Score>0] #[frame.binary_score=="Good"]
X=frame_pos[["Time_To_Read","Time_To_Complete","ratio"]]
y=frame_pos["binary_score"]

predicted_y= p.predict(X.values)
#print y 
#print predicted_y
print p.score(X.values,names.transform(y.values) ) 
print len(y.values)



#print p.coef_
#print p.get_params()
predicted_prob= p.predict_proba(X.values)
#print r.feature_importances_
ground_truth_y_predicted_prob=[]
print "classes"
print r.classes_
for i,j in enumerate(y):
    
    location= names.transform([j])[0]
    #print "name"
    #print j 
    #print location
    #print r.classes_
    #print predicted_prob[i]
    #print predicted_prob[i][location]
    ground_truth_y_predicted_prob.append(predicted_prob[i][location])
#good set
predicted=p.predict(X)
#print np.array(ground_truth_y_predicted_prob)
prc= precision_recall_curve(names.transform(y.values),
                       ground_truth_y_predicted_prob)
print len(prc[0])
print len(prc[1])
print len(prc[2])
print pandas.DataFrame(data={"pr":prc[0][:-1],"rc":prc[1][:-1],"thr":prc[2]})
confusion_matrix(names.transform(y.values),predicted)

0.655172413793
174
classes
[ 0.  1.]
86
86
85
          pr        rc       thr
0   0.635294  1.000000  0.288108
1   0.630952  0.981481  0.300502
2   0.638554  0.981481  0.307803
3   0.646341  0.981481  0.309975
4   0.641975  0.962963  0.335519
5   0.637500  0.944444  0.363713
6   0.645570  0.944444  0.370685
7   0.641026  0.925926  0.372216
8   0.636364  0.907407  0.374336
9   0.631579  0.888889  0.375807
10  0.626667  0.870370  0.376253
11  0.621622  0.851852  0.376464
12  0.616438  0.833333  0.385344
13  0.611111  0.814815  0.389431
14  0.605634  0.796296  0.392442
15  0.600000  0.777778  0.396140
16  0.594203  0.759259  0.403295
17  0.588235  0.740741  0.407396
18  0.582090  0.722222  0.418279
19  0.590909  0.722222  0.419545
20  0.584615  0.703704  0.424490
21  0.578125  0.685185  0.425077
22  0.587302  0.685185  0.439366
23  0.580645  0.666667  0.441579
24  0.573770  0.648148  0.442890
25  0.566667  0.629630  0.444154
26  0.559322  0.611111  0.450012
27  0.551724  0.592593  0.4537

array([[50, 16],
       [44, 64]])

In [279]:
#"Time_To_Read","Time_To_Complete","ratio"
time_to_read=30
time_to_complete=200
ratio=time_to_complete/time_to_read
print r.predict_proba([[time_to_read,time_to_complete,ratio]])
import cPickle
# save the classifier
with open('logistic_model_simple.pkl', 'wb') as fid:
    cPickle.dump(r, fid) 

#if score > 3
# 1- no_prob * score versus
#

[[ 0.45330046  0.54669954]]


In [60]:
from sklearn.model_selection import KFold
k=KFold(n_splits=2)
#r=tree.DecisionTreeClassifier(min_samples_split=10)
#p=make_pipeline(PolynomialFeatures(1), r)
frame_subset=frame #[frame["binary_score"]=="Bad"]
X=frame_subset[["Time_To_Read","Time_To_Complete","ratio"]]
y=frame_subset[["binary_score"]]
for train,test in k.split(X.values,y.values):
    train_X=X.values[train]
    train_y=y.values[train]
    train_X,train_y=balanced_subsample(train_X,names.transform(train_y))
    test_y=names.transform(y.values[test])
    test_X=X.values[test]
    p.fit(train_X,train_y)
    print p.score(test_X,test_y)
    print confusion_matrix(test_y,p.predict(test_X))

0.545454545455
[[13  5]
 [15 11]]
0.581395348837
[[10  5]
 [13 15]]


In [258]:
import pandas as pd 
import numpy as np 
ary=np.array(rating_score_matrix)

frame=pd.DataFrame(data=ary,columns=["Score","Avg_Score","Time_To_Read",
                                     "Time_To_Read_Result",
                                     "Time_To_Complete"])

frame["ratio"]=frame.Time_To_Complete/frame.Time_To_Read
frame["score_diff"]=(frame.Avg_Score-frame.Score).abs()
function=lambda t: "Bad" if t > 2 else "Good"
frame["binary_score"]=frame['score_diff'].apply(function)


names=preprocessing.LabelEncoder()
names.fit(frame['binary_score'])

LabelEncoder()

In [259]:
from sklearn.metrics import r2_score
from sklearn.linear_model import Ridge
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import PolynomialFeatures
from sklearn.pipeline import make_pipeline
from sklearn.metrics import mean_absolute_error
from sklearn import tree

r=tree.DecisionTreeClassifier(min_samples_split=10)
p=make_pipeline( r) #PolynomialFeatures(1),

#p=LogisticRegression()
frame=frame #[frame["score_diff"]>0]
X=frame[["Time_To_Read","Time_To_Read_Result","Time_To_Complete","ratio"]]
y=frame["binary_score"]

p.fit(X.values,y.values)

frame_pos=frame#[frame.binary_score=="Bad"]
X=frame_pos[["Time_To_Read","Time_To_Read_Result","Time_To_Complete","ratio"]]
y=frame_pos["binary_score"]
predicted_y= p.predict(X.values)
#print y 
#print predicted_y
#print p.score(X.values,y.values) 
#print mean_absolute_error(y,predicted_y)
#print mean_squared_error(y,predicted_y)
#print len(predicted_y)


#print pandas.DataFrame(data={"y_hat": predicted_y, "y":y})

In [260]:
from sklearn.model_selection import KFold
from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import SVR
from sklearn.preprocessing import StandardScaler
splits=2
k=KFold(n_splits=splits,shuffle=True)
r=RandomForestRegressor(min_samples_split=5)
r=Ridge()
#r=RandomForestRegressor(min_samples_split=5)
#r=SVR()#KNeighborsRegressor(n_neighbors=10)

#r=tree.DecisionTreeClassifier(min_samples_split=10)
#p=make_pipeline(PolynomialFeatures(1), r)
frame_subset=frame #[frame["binary_score"]=="Bad"]
X=frame_subset[["Score","ratio","Time_To_Read","Time_To_Read_Result","Time_To_Complete"]]
y=frame_subset["score_diff"]

X_transformed_scaler=StandardScaler()
X_transformed_scaler.fit(X.values)
p=make_pipeline(X_transformed_scaler, r)
#print X
total_score=0
total_abs_error=0
total_squared_err=0

#pandas.DataFrame(data={"x":X["ratio"],"y":y}).to_csv("graph_data.csv")

for train,test in k.split(X.values,y.values):
    train_X=X.values[train]
    train_y=y.values[train]
  
    test_y=y.values[test]
   
    test_X=X.values[test]
    
    #print len(test_y)
    #print len(train_y)
    p.fit(train_X,train_y)
    #print r.decision_path(test_X)
    total_score+= p.score(test_X,test_y)
   
    predicted_y=p.predict(test_X)
    
    print p.score(test_X,test_y)
   
    total_abs_error+= mean_absolute_error(test_y,predicted_y)
    total_squared_err+= mean_squared_error(test_y,predicted_y)
#print r.feature_importances_
print total_score/float(splits),
print total_abs_error/float(splits),
print total_squared_err/float(splits)

-0.0221985591442
-0.0607216945362
-0.0414601268402 1.07077417578 2.15664599067


In [269]:
#["Score","ratio","Time_To_Read","Time_To_Read_Result","Time_To_Complete"]
p.fit(X.values,y.values )
p.predict([[5,2,15,14,15]])

array([ 1.06112549])

In [242]:
def balanced_subsample(x,y,subsample_size=1.0):

    class_xs = []
    min_elems = None

    for yi in np.unique(y):
        elems = x[(y == yi)]
        class_xs.append((yi, elems))
        if min_elems == None or elems.shape[0] < min_elems:
            min_elems = elems.shape[0]

    use_elems = min_elems
    if subsample_size < 1:
        use_elems = int(min_elems*subsample_size)

    xs = []
    ys = []

    for ci,this_xs in class_xs:
        if len(this_xs) > use_elems:
            np.random.shuffle(this_xs)

        x_ = this_xs[:use_elems]
        y_ = np.empty(use_elems)
        y_.fill(ci)

        xs.append(x_)
        ys.append(y_)

    xs = np.concatenate(xs)
    ys = np.concatenate(ys)

    return xs,ys